# Summary



### Imports etc.

In [4]:
import copy
import csv
import datetime
import math

import numpy as np
import pandas as pd
import os,sys
import time
print('ready')

ready


In [5]:
import fasttext
import json
import requests
import math
import re
from collections import defaultdict


## Load trained model

In [10]:
day_start = datetime.date(2020,4,27)
day_end = datetime.date(2020,4,28)
lang = 'wikidata'

mode = 'cbow' ## (if 1: skip-gram, else cbow)
size = 50 ## number of dimensions
window = 5 ## context window size
sample = 0.001 ## downsample high-frequency words
negative = 5 ##negative sampling (noise words)
min_count = 20 ## words with less occurrences in total will be ignored
epochs  = 5 ## number of iterations

PATH_file = '../output/models/'
# # ## one week
filename = os.path.join(
    PATH_file,
    'word2vec_%s_%s--%s_params-%s-%s-%s-%s-%s-%s-%s.bin'%(
        lang,str(day_start),str(day_end),
        mode,size,window,sample,negative,min_count,epochs
    )
)
model = fasttext.load_model(filename)

In [4]:
# # set1: xenophobia
# list_qid_seed = ['Q84318312']
# N_nn = 200 ## initial number of nearet neighbors
# list_wikis = ['enwiki']
# list_keywords = []## if labels contain these words, remove item
# N_max=100 ## how many pages to keep from candidates
# formatted = False ## if formatting for wikitext


## set2: pandemic, disease
list_qid_seed = ['Q81068910','Q84263196']
N_nn = 1000
list_wikis = ['enwiki','dewiki','frwiki','eswiki','ruwiki','zhwiki','ptwiki','arwiki','bnwiki','hiwiki']
list_keywords = ['covid','corona','cov'] ## if labels contain these words, remove item
N_max = 200
formatted = True

#### get nearest neighbors of seed-article and query titles/sitelinks

In [5]:
def getItems(list_qids,list_wikis=None, n_batch=30):
    '''
    for list of qids, get:
    - label
    - description
    - titles (sitelinks)
    if list_wikis is specified, only get titles for specific wikis (e.g. enwiki)
    
    process the qids in batches of n_batches (at most 50, but make smaller to be sure).
    use: https://www.mediawiki.org/wiki/Wikibase/API
    '''
    api_url_base = 'https://wikidata.org/w/api.php'
    ## split into batches
    list_qids_split = np.array_split(list_qids,math.ceil(len(list_qids)/n_batch))
    list_items = []
    i_qid=0    
    for list_qids_batch in list_qids_split:
        
        params = {
            'action':'wbgetentities',
            'props':'sitelinks|labels|descriptions',
            'languages':'en',
            'format' : 'json',
            'ids':'|'.join(list_qids_batch),
        }
        if list_wikis != None:
            params['sitefilter']:'|'.join(list_wikis)

        response = requests.get( api_url_base,params=params)
        result=json.loads(response.text)
        
        for qid in list_qids_batch:
            dict_item = {'qid':qid}

            ## titles via sitelinks
            dict_title = {} ## collect all titles of an item via sitelinks
            titles = result['entities'].get(qid,{}).get('sitelinks',{})
            for wiki, wiki_title in titles.items():
                title = wiki_title['title'].replace(' ','_')
                dict_title[wiki] = title
            dict_item['titles'] = dict_title
            
            ## labels: put '-' if not available (so far only en)
            label = result['entities'].get(qid,{}).get('labels',{}).get('en',{}).get('value','-')
            dict_item['label'] = label
            
            ## description: put '-' if not available (so far only en)
            description = result['entities'].get(qid,{}).get('descriptions',{}).get('en',{}).get('value','-')
            dict_item['description'] = description
            
            list_items += [dict_item]
            
            i_qid+=1    
    return list_items
            

def items_add_titles(list_items, list_wikis = ['enwiki']):
    '''
    
    add information about a list of qids:
    - label
    - description
    - article-titles (sitelinks) for a set of wikis
    '''
    
    list_qids = [h['qid'] for h in list_items]
    list_items_meta = getItems(list_qids, list_wikis=list_wikis)
    
    list_items_new = list_items
    for i_item, item in enumerate(list_items):
        dict_titles = {}
        for wiki in list_wikis:
            title =  list_items_meta[i_item]['titles'].get(wiki,'-')
            dict_titles[wiki] = title
        list_items_new[i_item]['titles'] = dict_titles
        list_items_new[i_item]['label'] = list_items_meta[i_item]['label']
        list_items_new[i_item]['description'] = list_items_meta[i_item]['description']
        
        ## add text
        text = list_items_new[i_item]['label']
        if text == '-':
            text_tmp =  list_items_new[i_item]['description']
            if text_tmp!='-':
                text = text_tmp          

        for wiki in list_wikis:
            if text != '-':
                break
            else:
                text_tmp = list_items_new[i_item]['titles'][wiki]
                if text_tmp!='-':
                    text = text_tmp+' [title-%s]'%(wiki)
        list_items_new[i_item]['text'] = text
    return list_items_new
    

In [6]:
dict_nn_score = defaultdict(list)
for qid in list_qid_seed:
    qid_nn = model.get_nearest_neighbors(qid,k=N_nn)
    for score_tmp,qid_tmp in qid_nn:
        dict_nn_score[qid_tmp] += [score_tmp] 
        
list_nn_qid_tmp = []
list_nn_score_tmp = []
for qid_tmp,score_tmp in dict_nn_score.items():
    list_nn_qid_tmp+=[qid_tmp]
    list_nn_score_tmp += [max(score_tmp)]
    
indsort = np.argsort(list_nn_score_tmp)[::-1][:N_nn]
list_nn_qid = [list_nn_qid_tmp[i] for i in indsort]
list_nn_score = [list_nn_score_tmp[i] for i in indsort]
list_items = [
    {'qid': list_nn_qid[i],
     'score': list_nn_score[i]
    } 
    for i in range(N_nn) 
]
list_items = items_add_titles(list_items,list_wikis = list_wikis)
print(list_items[:2])
print(len(list_items))


[{'qid': 'Q82069695', 'score': 0.8901513814926147, 'titles': {'enwiki': 'Severe_acute_respiratory_syndrome_coronavirus_2', 'dewiki': 'SARS-CoV-2', 'frwiki': 'Coronavirus_2_du_syndrome_respiratoire_aigu_sévère', 'eswiki': 'SARS-CoV-2', 'ruwiki': 'SARS-CoV-2', 'zhwiki': '严重急性呼吸系统综合征冠状病毒2', 'ptwiki': 'Coronavírus_da_síndrome_respiratória_aguda_grave_2', 'arwiki': 'فيروس_كورونا_المرتبط_بالمتلازمة_التنفسية_الحادة_الشديدة_النوع_2', 'bnwiki': 'গুরুতর_তীব্র_শ্বাসযন্ত্রীয়_রোগলক্ষণসমষ্টি_সৃষ্টিকারী_করোনাভাইরাস_২', 'hiwiki': '2019_नोवेल_कोरोनावायरस'}, 'label': 'SARS-CoV-2', 'description': 'strain of virus causing the ongoing pandemic of coronavirus disease 2019 (COVID-19)', 'text': 'SARS-CoV-2'}, {'qid': 'Q84263196', 'score': 0.8782671093940735, 'titles': {'enwiki': 'Coronavirus_disease_2019', 'dewiki': 'COVID-19', 'frwiki': 'Maladie_à_coronavirus_2019', 'eswiki': 'COVID-19', 'ruwiki': 'COVID-19', 'zhwiki': '2019冠状病毒病', 'ptwiki': 'COVID-19', 'arwiki': 'مرض_فيروس_كورونا_2019', 'bnwiki': 'করোনাভাই

## filter list: disambiguation, keywords

In [7]:
def items_filter_notext(
    list_items):
    '''filter all items that have
    - no English label
    - no description
    - no existing title in any of the selected wikis
    '''
    list_items_new = []
    for item in list_items:
        if item['text'] != '-':
            list_items_new += [item]
    return list_items_new

def items_filter_disambiguation(
    list_items,
    list_keywords=['disambiguation']):
    s = re.compile('|'.join(list_keywords))
    list_items_new = []
    for item in list_items:
        label = item['label']+' '+item['description']
        if s.search(label.lower()) is None:
            list_items_new += [item]
    return list_items_new

def items_filter_keywords(
    list_items,
    list_keywords=[] ):
    if len(list_keywords) == 0:
        return list_items
    else:
        s = re.compile('|'.join(list_keywords))
        list_items_new = []
        for item in list_items:
            label = item['label']
            if s.search(label.lower()) is None:
                list_items_new += [item]
        return list_items_new

In [8]:
## filter if the label contains a keywords as substring
list_items_filtered = list(list_items)
list_items_filtered = items_filter_notext(list_items_filtered)
list_items_filtered = items_filter_disambiguation(list_items_filtered)
list_items_filtered = items_filter_keywords(list_items_filtered,list_keywords = list_keywords)
list_items = list(list_items_filtered)
print(list_items[:2])
print(len(list_items))

[{'qid': 'Q83554667', 'score': 0.8070012331008911, 'titles': {'enwiki': 'COVID-19_pandemic_lockdown_in_Hubei', 'dewiki': '-', 'frwiki': '-', 'eswiki': 'Encierro_de_Hubei_de_2020', 'ruwiki': '-', 'zhwiki': '2019冠狀病毒病中國大陸疫區封鎖措施', 'ptwiki': 'Lockdown_em_Hubei_em_2020', 'arwiki': 'عزل_ووهان_2020', 'bnwiki': '-', 'hiwiki': '-'}, 'label': '2020 Hubei lockdowns', 'description': 'Chinese quarantine effort in Hubei Province in response to the Wuhan coronavirus outbreak', 'text': '2020 Hubei lockdowns'}, {'qid': 'Q30314010', 'score': 0.7878186702728271, 'titles': {'enwiki': 'Social_distancing', 'dewiki': 'Räumliche_Distanzierung', 'frwiki': 'Distanciation_sociale', 'eswiki': 'Distanciamiento_físico', 'ruwiki': 'Социальное_дистанцирование', 'zhwiki': '保持社交距離', 'ptwiki': 'Distanciamento_social', 'arwiki': 'تباعد_اجتماعي', 'bnwiki': 'সামাজিক_দূরত্ব_স্থাপন', 'hiwiki': 'सामाजिक_दूरीकरण'}, 'label': 'social distancing', 'description': 'reduction of human social interaction in an effort to prevent the s

## get pageviews

In [9]:
def getRedirects(page,project):
    '''
    get all redirects (upto 500)
    for a given page 
    '''
    base_url = 'https://%s.org/w/api.php?action=query&titles=%s&prop=redirects&rdlimit=500&format=json' % (project,page)
    data = [p['title'] for p in list(requests.get(base_url).json()['query']['pages'].values())[0]['redirects']]
    return data
    

def getViews(page,start,end,project):
    """
    get pageviews using this API https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews 
    page: str (article name)
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)

    """
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/%s/all-access/all-agents/%s/daily/%s/%s" % (project,page,start,end)
    try:
        data = requests.get(base_url).json()['items']
        df = pd.DataFrame(data) [['views','timestamp']]
        df.rename(columns={'views':page},inplace=True)
    except KeyError:
        ## no pageviews- we have to set 1 date with 0 counts
        df = pd.DataFrame(columns=[page,'timestamp'],index=[0])
        df.iloc[0,0] = 0
        df.iloc[0,1] = start
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H')
    df.set_index('timestamp',inplace=True)
    return df

def getViewsMultiples(pages,start,end,project):
    """
    Get page views for a list of pages 
    pages: list (list of article's titles) ex: ['Chile','Brasil','Argentina']
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    results = []
    for page in pages:
        try:
            results.append(getViews(page,start,end,project))
        except:
            pass
    return pd.concat(results,axis=1)

def getViewsWithRedirects(page,start,end,project):
    """
    Get all redirects going to 'page' and get pageviews for that article
    page: str (article's title)
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    #start list of page
    pages = [page]
    try:
        #get all redirects to page 
        redirects = getRedirects(page=page,project=project)     
    except:
        #if getRedirects gives an error, we assume that there no pages redirecting to page
        redirects = []
    pages.extend(redirects)
    ## get pages views for all articles
    results = getViewsMultiples(pages=pages,start=start,end=end,project=project)
    #sum all pages views
    results = pd.DataFrame(results.sum(axis=1))
    results.rename(columns={0:page},inplace=True)
    return results      

def items_add_pageviews(list_items,start,end,N_max = 10):
    '''
    for a list of records [{'titles':{'enwiki':'Germany','dewiki':'Deutschland'}}, {}]
    add pageviews
    '''
    
    for i_item,item in enumerate(list_items[:N_max]):
        dict_pageviews = {}
        for wiki in list_wikis:
            project = '%s.wikipedia' % wiki.replace('wiki','')
            page = item['titles'][wiki]
            if page == '-':
                views = np.nan
            else:
                ## get pageviews per day
                views_df = getViewsWithRedirects(page,start,end,project)
                views = int(views_df.sum())
            ## sum pageviews
            dict_pageviews[wiki] = views
        list_items[i_item]['pageviews'] = dict_pageviews
    return list_items[:N_max]

In [10]:
start = str(day_start).replace('-','')
end = str(day_start+datetime.timedelta(days=n_days)).replace('-','')
list_items = items_add_pageviews(list_items,start,end,N_max=N_max)
print(list_items[:2])
print(len(list_items))

[{'qid': 'Q83554667', 'score': 0.8070012331008911, 'titles': {'enwiki': 'COVID-19_pandemic_lockdown_in_Hubei', 'dewiki': '-', 'frwiki': '-', 'eswiki': 'Encierro_de_Hubei_de_2020', 'ruwiki': '-', 'zhwiki': '2019冠狀病毒病中國大陸疫區封鎖措施', 'ptwiki': 'Lockdown_em_Hubei_em_2020', 'arwiki': 'عزل_ووهان_2020', 'bnwiki': '-', 'hiwiki': '-'}, 'label': '2020 Hubei lockdowns', 'description': 'Chinese quarantine effort in Hubei Province in response to the Wuhan coronavirus outbreak', 'text': '2020 Hubei lockdowns', 'pageviews': {'enwiki': 29236, 'dewiki': nan, 'frwiki': nan, 'eswiki': 1135, 'ruwiki': nan, 'zhwiki': 3538, 'ptwiki': 400, 'arwiki': 133, 'bnwiki': nan, 'hiwiki': nan}}, {'qid': 'Q30314010', 'score': 0.7878186702728271, 'titles': {'enwiki': 'Social_distancing', 'dewiki': 'Räumliche_Distanzierung', 'frwiki': 'Distanciation_sociale', 'eswiki': 'Distanciamiento_físico', 'ruwiki': 'Социальное_дистанцирование', 'zhwiki': '保持社交距離', 'ptwiki': 'Distanciamento_social', 'arwiki': 'تباعد_اجتماعي', 'bnwiki':

## add topics

In [39]:
def getTopic(qid,p_min=0.1):
    api_url_base = 'https://tools.wmflabs.org/wiki-topic/api/v1/wikidata/topic'
    params = {
        'qid':qid,
         'threshold':0.1
    }
    ## query api: https://tools.wmflabs.org/wiki-topic/
    response = requests.get( api_url_base,params=params)
    result=json.loads(response.text)   
    ## get scores (topic-probabilities) and topics
    scores = [h['score'] for h in result['results']]
    topics = [h['topic'] for h in result['results']]
    ## get topic w maximum probability (if p exceeds p_min)
    ind_max = np.argmax(scores)
    score_max = scores[ind_max]
    if score_max >= p_min:
        topic = topics[ind_max]
    else:
        topic = '-'
    return topic
    
def items_addTopics(list_items):
    for i_item,item in enumerate(list_items):
        qid = item['qid']
        topic = getTopic(qid)
        list_items[i_item]['topic'] = topic
    return list_items

In [40]:
list_items = items_addTopics(list_items)
print(list_items[:2])
print(len(list_items))

[{'qid': 'Q83554667', 'score': 0.8070012331008911, 'titles': {'enwiki': 'COVID-19_pandemic_lockdown_in_Hubei', 'dewiki': '-', 'frwiki': '-', 'eswiki': 'Encierro_de_Hubei_de_2020', 'ruwiki': '-', 'zhwiki': '2019冠狀病毒病中國大陸疫區封鎖措施', 'ptwiki': 'Lockdown_em_Hubei_em_2020', 'arwiki': 'عزل_ووهان_2020', 'bnwiki': '-', 'hiwiki': '-'}, 'label': '2020 Hubei lockdowns', 'description': 'Chinese quarantine effort in Hubei Province in response to the Wuhan coronavirus outbreak', 'text': '2020 Hubei lockdowns', 'pageviews': {'enwiki': 29236, 'dewiki': nan, 'frwiki': nan, 'eswiki': 1135, 'ruwiki': nan, 'zhwiki': 3538, 'ptwiki': 400, 'arwiki': 133, 'bnwiki': nan, 'hiwiki': nan}, 'topic': 'Geography.Regions.Asia.Asia*'}, {'qid': 'Q30314010', 'score': 0.7878186702728271, 'titles': {'enwiki': 'Social_distancing', 'dewiki': 'Räumliche_Distanzierung', 'frwiki': 'Distanciation_sociale', 'eswiki': 'Distanciamiento_físico', 'ruwiki': 'Социальное_дистанцирование', 'zhwiki': '保持社交距離', 'ptwiki': 'Distanciamento_soci

## Save data

In [41]:
df = pd.DataFrame()
if formatted == True:
    df['Wikidata-item'] = [ '[[wikidata:{0}|{0}]]'.format(h['qid']) for h in list_items ]
else:
    df['Wikidata-item'] = [ '{0}'.format(h['qid']) for h in list_items ]
df['label'] = [ h['text'] for h in list_items ]
df['score'] = [ '%.3f'%(h['score']) for h in list_items ]
df['topic'] = [ h['topic'] for h in list_items ]
for wiki in list_wikis:
    if formatted == True:
        df['%s:title'%(wiki)] = [  '[[%s:%s|%s]]'%(wiki[:-4],h['titles'][wiki],h['titles'][wiki].replace('_',' ')) if h['titles'][wiki]!='-' else '-'  for h in list_items ]
    else:
        df['%s:title'%(wiki)] = [  '%s'%(h['titles'][wiki].replace('_',' ')) if h['titles'][wiki]!='-' else '-'  for h in list_items ]

    df['%s:views'%wiki] = [ '%s'%(h['pageviews'][wiki]) if (h['pageviews'][wiki])>=500 else '<500' if np.isnan((h['pageviews'][wiki]))==False else '-'   for h in list_items ]

df.index = np.arange(1, len(df) + 1)
df.head()

,Wikidata-item,label,score,topic,enwiki:title,enwiki:views,dewiki:title,dewiki:views,frwiki:title,frwiki:views,...,zhwiki:title,zhwiki:views,ptwiki:title,ptwiki:views,arwiki:title,arwiki:views,bnwiki:title,bnwiki:views,hiwiki:title,hiwiki:views
1,[[wikidata:Q83554667|Q83554667]],2020 Hubei lockdowns,0.807,Geography.Regions.Asia.Asia*,[[en:COVID-19_pandemic_lockdown_in_Hubei|COVID...,29236,-,-,-,-,...,[[zh:2019冠狀病毒病中國大陸疫區封鎖措施|2019冠狀病毒病中國大陸疫區封鎖措施]],3538,[[pt:Lockdown_em_Hubei_em_2020|Lockdown em Hub...,<500,[[ar:عزل_ووهان_2020|عزل ووهان 2020]],<500,-,-,-,-
2,[[wikidata:Q30314010|Q30314010]],social distancing,0.788,STEM.STEM*,[[en:Social_distancing|Social distancing]],50533,[[de:Räumliche_Distanzierung|Räumliche Distanz...,4459,[[fr:Distanciation_sociale|Distanciation socia...,13688,...,[[zh:保持社交距離|保持社交距離]],1437,[[pt:Distanciamento_social|Distanciamento soci...,871,[[ar:تباعد_اجتماعي|تباعد اجتماعي]],629,[[bn:সামাজিক_দূরত্ব_স্থাপন|সামাজিক দূরত্ব স্থা...,602,[[hi:सामाजिक_दूरीकरण|सामाजिक दूरीकरण]],<500
3,[[wikidata:Q103177|Q103177]],severe acute respiratory syndrome,0.785,Compilation.List_Disambig,[[en:Severe_acute_respiratory_syndrome|Severe ...,96443,[[de:Schweres_akutes_Atemwegssyndrom|Schweres ...,11759,[[fr:Syndrome_respiratoire_aigu_sévère|Syndrom...,9816,...,[[zh:嚴重急性呼吸系統綜合症|嚴重急性呼吸系統綜合症]],7870,[[pt:Síndrome_respiratória_aguda_grave|Síndrom...,9713,[[ar:متلازمة_تنفسية_حادة_وخيمة|متلازمة تنفسية ...,4472,[[bn:গুরুতর_তীব্র_শ্বাসযন্ত্রীয়_রোগলক্ষণসমষ্ট...,<500,[[hi:सिवियर_एक्यूट_रेस्पिरेटरी_सिंड्रोम|सिवियर...,983
4,[[wikidata:Q12184|Q12184]],pandemic,0.770,STEM.STEM*,[[en:Pandemic|Pandemic]],139758,[[de:Pandemie|Pandemie]],59428,[[fr:Pandémie|Pandémie]],22840,...,[[zh:瘟疫|瘟疫]],7885,[[pt:Pandemia|Pandemia]],27378,[[ar:جائحة|جائحة]],6682,[[bn:বৈশ্বিক_মহামারী|বৈশ্বিক মহামারী]],791,[[hi:विश्वमारी|विश्वमारी]],4254
5,[[wikidata:Q84497471|Q84497471]],Li Wenliang,0.756,Culture.Biography.Biography*,[[en:Li_Wenliang|Li Wenliang]],33321,[[de:Li_Wenliang|Li Wenliang]],4772,[[fr:Li_Wenliang|Li Wenliang]],1909,...,[[zh:李文亮|李文亮]],7760,[[pt:Li_Wenliang_(médico)|Li Wenliang (médico)]],983,[[ar:لي_وين_ليانغ|لي وين ليانغ]],526,[[bn:লি_ওয়েনলিয়াং|লি ওয়েনলিয়াং]],<500,-,-


In [42]:
filename_save = os.path.join(
    os.pardir,
    'output',
    'lists'
    'reading-sessions-covid_list-%s_N%s_%s_%s.csv'%(
        '-'.join(list_qid_seed),
        N_max,
        str(day_start),
        str(day_start+datetime.timedelta(days=n_days-1))
    )
)
df.to_csv(filename_save)

In [44]:
print('success')

success
